## imports for Python, Pandas

In [14]:
import pandas as pd
import json
from pandas.io.json import json_normalize

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

Question 1:  country name isn't a nested field, so just read it in as a table and count them up



In [11]:
wbdata=pd.read_json('world_bank_projects.json')  #read data in as a table
freq=wbdata.countryshortname.value_counts()      # count and sort in descending order entries in column 'countryshortname'
freq[0:10]                                       #print first 10 entries in freq, the 10 most common countries


China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64

#Question 2
#mjtheme_namecode is a nested field, so need to use json_normalize first
#json_normalize only works on string data, so need to load data in as string first



In [12]:
mdata=json.load((open('world_bank_projects.json')))  #load data as string
themeList=json_normalize(mdata, 'mjtheme_namecode') #flatten the nested mjtheme_namecode field
freq=themeList.name.value_counts() #count instances of each kind of project theme
freq[0:10]                      #print first 10 entries in freq, the 10 most common themes
#Notice that the 7th most common theme is a blank field, as predicted



Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

Question #3: There are probably better ways to do this, but I'm going to make myself a little dictionary/lookup table code->names and use map to convert codes into names

I'll need to start with the same two steps as #2, opening the file and flattening the field.

In [13]:
mdata=json.load((open('world_bank_projects.json')))  #open data as a string
themeList=json_normalize(mdata, 'mjtheme_namecode') #flatten the nested mjtheme_namecode field

mydict={}  #initialize empty dictionary
for row in themeList.itertuples():    #loop over rows to make a dictionary
    if row.name!='' and not(row.code in mydict):  #if the name isn't blank and I don't already have this code
      mydict[row.code]=(row.name)           #add the code, name combination to the dictionary
        
#now use mydict to fill in names in the mjtheme_namecode name field.  
#first assign them the code #s, then map to names
themeList.name=themeList.code.values
themeList.name=themeList.name.map(mydict)

#exercise is done.  To check, repeat question #2 but with this new, complete list of theme names
freq=themeList.name.value_counts()      #count instances of each kind of project theme
freq[0:10]   
#Aha!  the blank name line is gone, numbers are updated, and we have the new name of 'Economic development'

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64